In [1]:
import pandas as pd
import herepy
from tqdm.notebook import tqdm
from collections import defaultdict
import re
import requests
import urllib
import xmltodict
import json
import numpy as np
#packages for R in python
import rpy2.robjects as ro
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr
#CPD
from pgmpy.models import BayesianModel
import pgmpy
# from pgmpy.estimators import ConstraintBasedEstimator
#bnlearn
import bnlearn
import torch
import pyro
import pyro.distributions as dist
from statistics import mean
from pyro.infer import Importance, EmpiricalMarginal
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import Image

%matplotlib inline

In [2]:
listings = pd.read_csv('data/listings.csv')
listings['is_location_exact'].unique()

array(['t', 'f'], dtype=object)

In [3]:
listings = listings[listings['is_location_exact'] == 't']

In [4]:
listings = listings[listings['property_type'].isin(['Apartment','Condominium'])]

In [5]:
listings = listings[listings['room_type']=='Entire home/apt']

In [6]:
listings = listings[listings['calendar_updated'].isin(['today','yesterday','2 days ago',
                                                       '3 days ago','4 days ago','5 days ago',
                                                       '6 days ago','a week ago','1 week ago',
                                                       '2 weeks ago','3 weeks ago','4 weeks ago',
                                                       '5 weeks ago','6 weeks ago','7 weeks ago',
                                                       '2 months ago','3 months ago','4 months ago',
                                                       '5 months ago','6 months ago','7 months ago',
                                                       '8 months ago','9 months ago','10 months ago',
                                                       '11 months ago','12 months ago'])]

In [7]:
listings = listings[listings['first_review'].notnull()]

In [11]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [39]:
def herePy(coordinate1,coordinate2):
    coordinates = coordinate1+','+coordinate2
    locator = Nominatim(user_agent="myGeocoder")
    address = locator.reverse(coordinates)
    address = address.raw
    return pd.Series([address.get('display_name',None),
                      address['address'].get('house_number',address['address'].get('building',None)),
                      address['address'].get('road',None),
                      address['address'].get('country',None),
                      address['address'].get('postcode',None)],
                     index=['Address','House','Street','Country',
                            'PostalCode'])

In [41]:
# locator = Nominatim(user_agent="myGeocoder")
# coordinates = '42.3547464,-71.0740765'
# location = locator.reverse(coordinates)
# location.raw
herePy('42.3547464','-71.0740765')

Address       135, Beacon Street, Beacon Hill, Boston, Suffo...
House                                                       135
Street                                            Beacon Street
Country                                United States of America
PostalCode                                                02116
dtype: object

In [42]:
listings[['Address','HouseNumber','Street','Country','PostalCode']] = listings.progress_apply(lambda row: herePy(str(row['latitude']), str(row['longitude'])) , axis=1)

In [43]:
lisiting_manual = pd.read_csv('data/listings_manual.csv')
listings_full   = lisiting_manual.merge(listings,on='id')

In [44]:
no_unique_columns = [x for y,x in enumerate(listings_full.columns) if len(set(list(listings_full[x])))==1]
listings_full = listings_full.drop(columns=no_unique_columns).reset_index(drop=True)

In [45]:
i = (0.04/12)   # Interest rate divided by 12
dr = 0.80       # Principal for mortgage
n  = 15*12      # 15 years mortgage
multiplier = (i*((1+i)**n))/(((1+i)**n)-1) # compound interest formula
listings_full['mortgage_p_month'] = (listings_full['Zestimate'].astype(float)*multiplier*dr).astype(int)

In [46]:
listings_full['annual_tax']  = (listings_full['Zestimate'].astype(float)*0.00804).astype(int)

In [47]:
listings_full['ins_p_month'] = (listings_full['Zestimate'].astype(float)*0.00035).astype(int)

In [48]:
listings_full['price'] = listings_full['price'].str.replace('$','')
listings_full['price'] = listings_full['price'].str.replace(',','').astype(float)

In [50]:
listings_full['roi'] = (listings_full['price']*340 - (
    (listings_full['mortgage_p_month']+ listings_full['ins_p_month'])*12 +
    listings_full['mortgage_p_month']))/(0.2*listings_full['Zestimate'].astype(float))*100

In [51]:
# Function to query `walkscore.com` to get the score for walking, transit and biking score
def walkScore(address,lat,lon,api,transit=1,bike=1):
    add  = re.sub('\s', '%', address)
    latitude = "&lat="+str(lat)
    longitude= "&lon="+str(lon)
    tran_code= "&transit="+str(transit)
    bike_code= "&bike="+str(bike)
    api_key  = "&wsapikey="+api
    api_url  = "http://api.walkscore.com/score?format=json&address="
    r = requests.get(api_url+add+latitude+longitude+tran_code+bike_code+api_key).json()
    walkScore = r['walkscore']
    bikeScore = r['bike']['score']
    trasitScore = r['transit']['score']
    return pd.Series([walkScore,bikeScore,trasitScore],index=['walkScore','bikeScore','transitScore'])

In [54]:
walkscoreAPI = 'b12d2c039749ebd2e8f67ffe0d9f2b83'
tqdm.pandas()
listings_full[['walkScore','bikeScore','transitScore']] = listings_full.progress_apply(
    lambda row:walkScore(row['Address'],row['latitude'], row['longitude'],api=walkscoreAPI), axis=1)

/Users/abdulrehman/opt/anaconda3/envs/Airbnb_Analysis/lib/python3.8/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [55]:
def schoolsNearMe(lat,lon,radius,greatSchoolsAPi):
    url_base  = "https://api.greatschools.org/schools/nearby?key="
    api       = greatSchoolsAPi+"&state=MA"
    lat_url   = "&lat="+str(lat)
    lon_url   = "&lon="+str(lon)
    radius_url= "&radius="+str(radius)

    url       =  url_base+api+lat_url+lon_url+radius_url

    contents = urllib.request.urlopen(url).read()
    my_dict=xmltodict.parse(contents)
    json_data=json.dumps(my_dict)
    json_data2 = json.loads(json_data)
    
    lister = [int(x['gsRating']) for x in json_data2['schools']['school'] if 'gsRating' in x.keys()]

    score = sum(lister)/len(lister)
    return pd.Series([score],index=['publicSchoolScores '])

In [ ]:
greatSchoolsAPi= #Enter API
listings_full[['publicSchoolScores']] = listings_full.progress_apply(lambda row: schoolsNearMe(row['latitude'], row['longitude'],radius=2,greatSchoolsAPi) , axis=1)

In [56]:
listings_full = pd.read_csv('data/listings_full.csv')

In [57]:
listings_subset = listings_full[['id','Zestimate','bedrooms','bathrooms','beds','accommodates','amenities_score',
               'bikeScore','walkScore','transitScore','neighbourhood_cleansed','property_type',
               'publicSchoolScores','closenessToDowntown','price','minimum_nights_avg_ntm','roi']]
pd.options.mode.chained_assignment = None 
listings_subset['property_type'] = listings_subset['property_type'].astype('category')
listings_subset['bathrooms'] = listings_subset['bathrooms'].astype('category')
listings_subset['neighbourhood'] = listings_subset['neighbourhood_cleansed'].astype('category')
del listings_subset['neighbourhood_cleansed']

listings_subset['roi'] = listings_subset['roi'].astype('float')
listings_subset['Zestimate'] = listings_subset['Zestimate'].astype('float')
listings_subset['bedrooms'] = listings_subset['bedrooms'].astype('float')
listings_subset['beds'] = listings_subset['beds'].astype('float')
listings_subset['accommodates'] = listings_subset['accommodates'].astype('float')
listings_subset['amenities_score'] = listings_subset['amenities_score'].astype('float')
listings_subset['publicSchoolScores'] = listings_subset['publicSchoolScores'].astype('float')
listings_subset['closenessToDowntown'] = listings_subset['closenessToDowntown'].astype('float')
listings_subset['price'] = listings_subset['price'].astype('float')
listings_subset['minimum_nights_avg_ntm'] = listings_subset['minimum_nights_avg_ntm'].astype('float')
pd.options.mode.chained_assignment = 'warn'

In [58]:
listing_cat = listings_subset[['bathrooms','bikeScore','walkScore','transitScore','property_type','neighbourhood']]
listings_n_cat = listings_subset[['roi','Zestimate','bedrooms','beds','accommodates','amenities_score','publicSchoolScores','closenessToDowntown','price','minimum_nights_avg_ntm']]

In [60]:
def rDiscretize(data):
    utils = importr('utils')
    utils.install_packages('bnlearn')
    # utils.install_packages('base')
    bnlearnR = importr('bnlearn')
    bnDiscretize = bnlearnR.discretize
    # Import Functions
    pandas2ri.activate()
    r_dt = ro.conversion.py2rpy(data) #convert pandas to RDataframe
    r_dt2 = bnDiscretize(r_dt,method= "hartemink", breaks = 3, ibreaks=3)
    pd_dt = ro.conversion.rpy2py(r_dt2)
    pd_dt.reset_index(drop=True)
    return pd_dt

In [65]:
listings_fac = rDiscretize(listings_n_cat)

--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Australia (Perth) [https]
 6: Austria [https]
 7: Belgium (Ghent) [https]
 8: Brazil (BA) [https]
 9: Brazil (PR) [https]
10: Brazil (RJ) [https]
11: Brazil (SP 1) [https]
12: Brazil (SP 2) [https]
13: Bulgaria [https]
14: Canada (MB) [https]
15: Chile (Santiago) [https]
16: China (Beijing 2) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Guangzhou) [https]
20: China (Lanzhou) [https]
21: China (Nanjing) [https]
22: China (Shanghai 1) [https]
23: China (Shanghai 2) [https]
24: Costa Rica [https]
25: Denmark [https]
26: East Asia [https]
27: Ecuador (Cuenca) [https]
28: Ecuador (Quito) [https]
29: Estonia [https]
30: France (Lyon 1) [https]
31: France (Lyon 2) [https]
32: France (Marseille) [https]
33: France (Montpellier) [https]
34: Germany (Erlangen

R[write to console]: trying URL 'https://cran.case.edu/bin/macosx/el-capitan/contrib/3.6/bnlearn_4.6.1.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2653896 bytes (2.5 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R


The downloaded binary packages are in
	/var/folders/40/1cmwtr_5693092wp0mnttzgh0000gn/T//RtmpCjcHUt/downloaded_packages


In [67]:
data_discrete = listings_fac.reset_index(drop=True).join(listing_cat)

In [68]:
nodes = ['amenities_score','bathrooms','bedrooms','transitScore','Zestimate','neighbourhood',
         'roi','price','minimum_nights_avg_ntm','bikeScore','walkScore','publicSchoolScores',
         'property_type','closenessToDowntown','accommodates','beds']

In [69]:
edges = [('amenities_score','price'),
         ('bathrooms','price'),
         ('bathrooms','Zestimate'),
         ('bedrooms','price'),
         ('bedrooms','beds'),
         ('bedrooms','accommodates'),
         ('bedrooms','Zestimate'),
         ('property_type','price'),
         ('property_type','Zestimate'),
         ('price','roi'),
         ('transitScore','Zestimate'),
         ('transitScore','price'),
         ('Zestimate', 'price'),
         ('Zestimate','roi'),
         ('bikeScore','price'),
         ('walkScore','price'),
         ('beds','price'),
         ('accommodates','price'),
         ('neighbourhood','price'),
         ('neighbourhood','transitScore'),
         ('neighbourhood','publicSchoolScores'),
         ('publicSchoolScores','Zestimate'),
         ('neighbourhood','Zestimate'),
         ('neighbourhood','closenessToDowntown'),
         ('closenessToDowntown','price'),                    
         ('minimum_nights_avg_ntm','price')]

In [70]:
from pgmpy.models import BayesianModel
graph = BayesianModel()

graph.add_nodes_from(nodes)
graph.add_edges_from(edges)

In [71]:
def adjacencyMatrix(edges):
    df = pd.DataFrame(edges, columns =['source','target'], dtype = float) 
    df = pd.crosstab(df.source, df.target)
    idx = df.columns.union(df.index)
    df2 = df.reindex(index = idx, columns=idx, fill_value=0)
    df2.index.name = "source"
    df2 = df2.rename_axis('target')
    return df2

In [72]:
adjMat = adjacencyMatrix(edges)

In [73]:
adjMat

source,Zestimate,accommodates,amenities_score,bathrooms,bedrooms,beds,bikeScore,closenessToDowntown,minimum_nights_avg_ntm,neighbourhood,price,property_type,publicSchoolScores,roi,transitScore,walkScore
target,,,,,,,,,,,,,,,,
Zestimate,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
accommodates,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
amenities_score,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
bathrooms,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
bedrooms,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
beds,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
bikeScore,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
closenessToDowntown,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
minimum_nights_avg_ntm,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [74]:
data_discrete = data_discrete.astype(object)

In [ ]:
model ={'adjmat':adjMat, 'model':graph}

model_update = bnlearn.parameter_learning.fit(model, data_discrete, verbose=1)